#  Tráfico de Madrid

El dataset ['trafico_madrid.csv'](./datos/trafico_madrid.csv) contiene datos de tráfico. Distintos sensores de control de tráfico de la Comunidad de Madrid proporcionan datos sobre la intensidad de tráfico cada 15 minutos.



__Descripción de los datos de tráfico__


* `idelem`: Identificación  única  del  Punto  de  Medida  en  los  sistemas  de  control  del tráfico del Ayuntamiento de Madrid. 
* `fecha`: fecha en formato Año, mes, día, hora, minutos, segundos
* `intensidad`: Intensidad   del   Punto   de   Medida   en
   el   periodo   de   15   minutos  (vehículos/hora). 
* `vmed`: Velocidad  media  de  los  vehículos  en  el periodo  de  15  minutos  (Km./h). Sólo para puntos de medida interurbanos M30. 
* `error`: Indicación de si ha habido al menos una
 muestra errónea o sustituida en el periodo de 15 minutos. 
    * `N`: no ha habido errores ni sustituciones 
    * `E`: los parámetros de calidad de alguna de las muestras integradas no son óptimos. 
* `periodo_integracion`: Número   de   muestras   recibidas   y   consideradas

__1. Crea un dataframe con los datos del fichero  ['trafico_madrid.csv'](trafico_madrid.csv).__

In [1]:
# Sol:
import pandas as pd
import numpy as np


In [2]:
df=pd.read_csv('./datos/trafico_madrid.csv')
df
df.head()

,idelem,fecha,identif,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion
0,3621,2017-02-18 07:30:00,17016,PUNTOS MEDIDA URBANOS,207,0,8.0,0,N,15
1,3973,2017-02-07 00:00:00,06033,PUNTOS MEDIDA URBANOS,134,1,6.0,0,N,13
2,4064,2017-02-11 22:45:00,10018,PUNTOS MEDIDA URBANOS,169,3,NaN,0,NaN,15
3,6375,2017-02-08 00:00:00,59021,PUNTOS MEDIDA URBANOS,53,1,7.0,0,N,14
4,6691,2017-02-23 20:45:00,PM12121,--,2428,5,37.0,81,NaN,15


__2.__ Cada estación de tráfico tiene unas coordenadas. Dicha información se encuentra en el fichero [./datos/pmed_trafico_coord.csv](./datos/pmed_trafico_coord.csv). __Abre dicho fichero para ver los datos que contiene. Usa la función `read_csv` para leer los datos y crear un dataframe.__ Solo nos interesan las columnas `['idelem', 'cod_cent', 'Xcoord', 'Ycoord']`. Visualiza las 5 primeras filas.

In [3]:
# Sol: 
coord=pd.read_table('./datos/pmed_trafico_coord.csv',
                   sep=';', 
                   usecols=['idelem','cod_cent', 'Xcoord', 'Ycoord'],
                   decimal=',')

coord
coord.head()

,idelem,cod_cent,Xcoord,Ycoord
0,1044,03FT08PM01,-3.720754,40.419125
1,3600,PM30901,-3.664409,40.406252
2,3705,PM41451,-3.709906,40.390721
3,6823,PM41453,-3.717737,40.384554
4,7033,1015,-3.690105,40.426231


__3. Añadir las columnas de las coordenadas de cada estación a los datos de tráfico del mes de Enero. Usa la función `merge`.__

In [4]:
# Sol: la columna de fechas hay que trabajarla
fichero="./datos/pmed_trafico_coord.csv"

import datetime as dt
df2=pd.read_csv(fichero,
                   usecols= [2,5,6],
                   sep= ';', decimal= ',')
df['fecha']=pd.to_datetime(df['fecha'])
                           
##convertimos a mes de enero y le damos el valor 1 (mes 1)
enero=df['fecha'].dt.month
mesdf=enero == 1
dfcoord=df[mesdf].merge(df2, on='idelem')
dfcoord

##sale un DF vacio pero todo está unido

,fecha,identif,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion,idelem,Xcoord,Ycoord


In [5]:
dfcoord.dtypes

#Xcoord y Ycoord son de tipo string

fecha                  datetime64[ns]
identif                        object
tipo_elem                      object
intensidad                      int64
ocupacion                       int64
carga                         float64
vmed                            int64
error                          object
periodo_integracion             int64
idelem                          int64
Xcoord                        float64
Ycoord                        float64
dtype: object

__4. Añadir una hora a la columna fecha. La función `to_timedelta` permite crear unidades de tiempo. Crear una hora y súmala a la columna fecha.__

In [6]:
# Sol: 
hora=pd.to_timedelta(1,'h')
hora
df['fecha']=pd.to_datetime(df['fecha'])

df['fecha']=df['fecha'] + hora

df['fecha'].head() #ver solo las 5 primeras entradas


0   2017-02-18 08:30:00
1   2017-02-07 01:00:00
2   2017-02-11 23:45:00
3   2017-02-08 01:00:00
4   2017-02-23 21:45:00
Name: fecha, dtype: datetime64[ns]

__5.__ Los datos de tráfico se recogen cada 15 minutos. Pero a nosotros __nos interesa agrupar los datos para tener una observación por estación de tráfico y hora__. En realidad queremos calcular la media de intensidad por hora. Usa el método `resample`.


In [7]:
# Sol: ##resample para calcular la media de intensidad por hora

observacion=df.reset_index().set_index('fecha').intensidad.resample('H').mean()
observacion.head()
#solucion=df.intensidad.resample('H').mean()
#sol


fecha
2017-02-01 01:00:00    163.430894
2017-02-01 02:00:00     84.253521
2017-02-01 03:00:00     49.172414
2017-02-01 04:00:00     41.034483
2017-02-01 05:00:00     39.702479
Freq: H, Name: intensidad, dtype: float64

__6. Calcula el número de centros asociados a cada `idelem`.__

In [8]:
# Sol: 
##primero combinar las tablas
df_coord=pd.merge(df,coord, on='idelem')
centros=df_coord.groupby('idelem').count() ##primero hacemos un groupby para tener los datos correctos de idelem y sobre esto 
centros['cod_cent'].head()

##es decir, en este caso, hay 15 centros en 1001.. etc

idelem
1001    15
1002    32
1003    34
1006    26
1009    27
Name: cod_cent, dtype: int64

__7. Para cada `idelem`, calcula el `identif` que registra la intensidad máxima.__

In [9]:
df=pd.read_csv('./datos/trafico_madrid.csv', parse_dates=[1], index_col=[1])
df.head()

,idelem,identif,tipo_elem,intensidad,ocupacion,carga,vmed,error,periodo_integracion
fecha,,,,,,,,,
2017-02-18 07:30:00,3621,17016,PUNTOS MEDIDA URBANOS,207,0,8.0,0,N,15
2017-02-07 00:00:00,3973,06033,PUNTOS MEDIDA URBANOS,134,1,6.0,0,N,13
2017-02-11 22:45:00,4064,10018,PUNTOS MEDIDA URBANOS,169,3,NaN,0,NaN,15
2017-02-08 00:00:00,6375,59021,PUNTOS MEDIDA URBANOS,53,1,7.0,0,N,14
2017-02-23 20:45:00,6691,PM12121,--,2428,5,37.0,81,NaN,15


In [12]:
# Sol: aquí piden que se devuelva un valor x(identif) segun el max del valor y(idelem) de una columna
df['maximo'] = df.groupby(['idelem'])['intensidad'].transform(max)
res1 = df['intensidad'] == df['maximo']
df[res1][['idelem','identif']].head()

##otra forma
#m={'intensidad': max}
#dfnuevo=df.groupby('idelem')
#dfnuevo['intensidad', 'identif'].agg(m).head()


,idelem,identif
fecha,,
2017-02-16 11:00:00,5784,40002
2017-02-02 13:00:00,3528,44402
2017-02-15 08:45:00,4659,35022
2017-02-06 19:30:00,5959,45013
2017-02-01 17:30:00,3789,76007


__8. Modifica las columnas de coordenadas para que aparezcan solo con 2 decimales.__

In [13]:
# Sol: #Formateamos 'Xcoord' y 'Ycoord'

##quitar los decimales 
coord['Xcoord']=coord['Xcoord'].round(2)
coord['Ycoord']=coord['Ycoord'].round(2)
coord.head()

#Otro método
#coord['Ycoord']=dfcoord['Ycoord'].astype(float) esto seria la conversion a float en otro caso
#tablita = t4.applymap(lambda x : round(x,2) if type(x) == float else x)
#tablita

,idelem,cod_cent,Xcoord,Ycoord
0,1044,03FT08PM01,-3.72,40.42
1,3600,PM30901,-3.66,40.41
2,3705,PM41451,-3.71,40.39
3,6823,PM41453,-3.72,40.38
4,7033,1015,-3.69,40.43


In [14]:
coord.dtypes

idelem        int64
cod_cent     object
Xcoord      float64
Ycoord      float64
dtype: object

__9. ¿En qué centro se ha registrado la mayor intensidad de tráfico?.__

In [15]:
# Sol: aquí piden que se devuelva un valor x(centro) segun el max del valor y(intensidad) de una columna
##vamos a simplificar las tablas

info=pd.read_csv(fichero, usecols=[1,2,3,4], sep=';',
                decimal=',')
df_centros=pd.merge(df, info, on='idelem')

info_centro=df_centros['intensidad'] == df['intensidad'].max()
df_centros[info_centro].nombre

##La mayor intensidad de trafico se ha registrado en entrada de la M20 en Menendez Alvaro num 46031 (en la tabla)

46031    Entrada M-30 N-S  - M-30 - Mindez rlvaro
Name: nombre, dtype: object